In [ ]:
import tensorflow as tf
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

from backend.preference_aggregation_featureless import FeaturelessPreferenceLearningModel
from backend.preference_aggregation_featureless import FeaturelessMedianPreferenceAverageRegularizationAggregator

from backend.toy_preference_dataset import ToyHardcodedDataset, ToyRandomDataset

from tqdm import tqdm

In [ ]:
dataset = ToyHardcodedDataset()
dataset._generate_many(100)

In [ ]:
models = [FeaturelessPreferenceLearningModel(objects=dataset.objects,
                                   output_dim=len(dataset.fields),
                                   name=user) for user in dataset.users]

In [ ]:
aggregator = FeaturelessMedianPreferenceAverageRegularizationAggregator(models=models)

In [ ]:
aggregator.save('.')

In [ ]:
aggregator.load('.')

In [ ]:
for r in dataset.ratings:
    u_idx = dataset.users.index(r['user'])
    ratings_as_vector = np.array([r['ratings'][k] for k in dataset.fields]) / 100.
    models[u_idx].register_preference(o1=r['o1'], o2=r['o2'], p1_vs_p2=ratings_as_vector)

In [ ]:
aggregator.fit(epochs=10000)

In [ ]:
aggregator.models[0]('trump_video')

In [ ]:
aggregator.common_model.scores

In [ ]:
aggregator('trump_video')

In [ ]:
def test_hardcoded_dataset(dataset, aggregator):
    """Test that results are valid on a hardcoded dataset."""
    for user_id, user in enumerate(dataset.users):
        got_scores = aggregator.models[user_id].scores
        expect_scores = dataset.scores_dict[user]
        errors = 0
        for i, feature in enumerate(dataset.fields):
            for o1 in dataset.objects:
                for o2 in dataset.objects:
                    if o1 == o2: continue
                    delta1 = got_scores[o2][i] - got_scores[o1][i]
                    if (o1, o2) in expect_scores[feature]:
                        delta2 = expect_scores[feature][(o1, o2)]
                    else:
                        delta2 = 100 - expect_scores[feature][(o2, o1)]
                    delta2 = (delta2 - 50) / 50.
                    if delta1 * delta2 <= 0:
                        print("Invalid result: %s %s %s %s" % (user, feature, o1, o2))
                        errors += 1
        assert not errors, "There were %s errors" % errors

In [ ]:
test_hardcoded_dataset(dataset, aggregator)

In [ ]:
aggregator.plot_loss()

In [ ]:
dataset = ToyRandomDataset()
dataset._generate_many(100)

In [ ]:
aggregator1(dataset.objects[0])